In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import recall_score
import optuna

pd.options.display.max_columns = 100
pd.options.display.max_rows = 50
optuna.logging.set_verbosity(0)

In [2]:
train = pd.read_csv('data/train.csv')
sample = pd.read_csv('data/sample_solution.csv')
test = pd.read_csv('data/test_dataset_test.csv')

cols_to_drop = ['ID', 'ID_y', 'Время засыпания']
train = train.drop(cols_to_drop, axis=1)

In [3]:
train['Сигарет в день'] = train['Сигарет в день'].fillna(0)
train['Возраст курения'] = train['Возраст курения'].fillna(0)
train['Возраст алког'] = train['Возраст алког'].fillna(0)

In [4]:
all_dicts = {}
all_dicts['Пол'] = LabelEncoder().fit(train['Пол'])
train['Пол'] = all_dicts['Пол'].transform(train['Пол'])

all_dicts['Семья'] = LabelEncoder().fit(train['Семья'])
train['Семья'] = all_dicts['Семья'].transform(train['Семья'])

all_dicts['Этнос'] = LabelEncoder().fit(train['Этнос'])
train['Этнос'] = all_dicts['Этнос'].transform(train['Этнос'])

all_dicts['Национальность'] = LabelEncoder().fit(train['Национальность'])
train['Национальность'] = all_dicts['Национальность'].transform(train['Национальность'])

all_dicts['Религия'] = LabelEncoder().fit(train['Религия'])
train['Религия'] = all_dicts['Религия'].transform(train['Религия'])

all_dicts['Профессия'] = LabelEncoder().fit(train['Профессия'])
train['Профессия'] = all_dicts['Профессия'].transform(train['Профессия'])

all_dicts['Образование'] = LabelEncoder().fit(train['Образование'])
train['Образование'] = all_dicts['Образование'].transform(train['Образование'])

all_dicts['Время пробуждения'] = LabelEncoder().fit(train['Время пробуждения'])
train['Время пробуждения'] = all_dicts['Время пробуждения'].transform(train['Время пробуждения'])


all_dicts['Статус Курения'] = LabelEncoder().fit(train['Статус Курения'])
train['Статус Курения'] = all_dicts['Статус Курения'].transform(train['Статус Курения'])

all_dicts['Частота пасс кур'] = LabelEncoder().fit(train['Частота пасс кур'])
train['Частота пасс кур'] = all_dicts['Частота пасс кур'].transform(train['Частота пасс кур'])

all_dicts['Алкоголь'] = LabelEncoder().fit(train['Алкоголь'])
train['Алкоголь'] = all_dicts['Алкоголь'].transform(train['Алкоголь'])

In [5]:
good_cols = ['Пол', 'Вы работаете?', 'Выход на пенсию',
       'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
       'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма',
       'Туберкулез легких ', 'ВИЧ/СПИД',
       'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы',
       'Статус Курения', 'Возраст курения', 'Сигарет в день',
       'Пассивное курение', 'Частота пасс кур', 'Алкоголь', 'Возраст алког']

In [6]:
target_cols = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']

features = train.drop(target_cols, axis=1)[good_cols]
target = train[target_cols].values

In [7]:
cat_f = ['Пол', 'Вы работаете?', 'Выход на пенсию',
       'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
       'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма',
       'Туберкулез легких ', 'ВИЧ/СПИД',
       'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы',
       'Статус Курения', 'Пассивное курение', 'Частота пасс кур', 'Алкоголь']

In [8]:
def objective(trial):
    borders = trial.suggest_float('threshold', 0.05, 0.95)
    score = recall_score(y_valid, (predictions > borders).astype('int'))
    return score

In [9]:
models = []
all_scores = []
for x in range(5):
    cur_target = target[:, x]

    strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cur_scores = []
    for fold, (train_index, valid_index) in enumerate(strat_kfold.split(features, cur_target)):
        X_train, y_train = features.iloc[train_index], cur_target[train_index]
        X_valid, y_valid = features.iloc[valid_index], cur_target[valid_index]

        model = CatBoostClassifier(
            depth=3,
            iterations=3000,
            random_seed=42,
            early_stopping_rounds=2000,
            eval_metric='Recall',
            logging_level='Silent'
        )

        model.fit(Pool(X_train, y_train, cat_features=cat_f), eval_set=Pool(X_valid, y_valid, cat_features=cat_f), verbose=200)
        predictions = model.predict_proba(X_valid)[:, 1]
        
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=100)
        threshold = study.best_params['threshold']

        valid_preds = (model.predict_proba(X_valid)[:, 1] > study.best_params['threshold']).astype('int')
        cur_scores.append(recall_score(y_valid, (model.predict_proba(X_valid)[:, 1] > study.best_params['threshold']).astype('int')))
    all_scores.append(np.mean(cur_scores))
    print(cur_scores)

all_scores

[1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0]
[0.9130434782608695, 0.8695652173913043, 0.9565217391304348, 1.0, 0.7916666666666666]
[1.0, 1.0, 1.0, 0.9473684210526315, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0]


[1.0, 1.0, 0.9061594202898551, 0.9894736842105264, 1.0]

In [9]:
train[train['Прочие заболевания сердца']==1]['Этнос'].value_counts()

1    86
Name: Этнос, dtype: int64

In [6]:
feature_imp = pd.DataFrame({'feature_imp': model.get_feature_importance(), 'feature': features.columns}).sort_values('feature_imp', ascending=False)
feature_imp

,feature_imp,feature
0,84.903772,Пол
25,10.227149,Алкоголь
1,4.869079,Семья
16,0.000000,ВИЧ/СПИД
29,0.000000,"Спорт, клубы"
28,0.000000,Сон после обеда
27,0.000000,Время пробуждения
26,0.000000,Возраст алког
24,0.000000,Частота пасс кур
23,0.000000,Пассивное курение
